In [1]:
%load_ext autoreload
%autoreload 2
import litebird_sim as lbs
import numpy as np
import healpy as hp
import sbm
import logging
import toml
from pathlib import Path
import matplotlib.pyplot as plt
from logging import getLogger, INFO, WARNING, ERROR, CRITICAL
logging.basicConfig(level=ERROR)


#logger = getLogger(__name__)
#logger.setLevel(ERROR)
# Load your Imo in litebird_sim

CONFIG_PATH = Path.home() / ".config" / "litebird_imo"
CONFIG_FILE_PATH = CONFIG_PATH / "imo.toml"
tomlinfo = toml.load(CONFIG_FILE_PATH)
flatfile_location = tomlinfo["repositories"][0]["location"]

imo_version = "v2"
imo = lbs.Imo(flatfile_location=flatfile_location)

In [2]:
#function to select two (closest to) boresight couple of detectors
def detector_list(instrument, channel):
    if (
        instrument == "MFT"
        and not channel[3:] == "119"
        and not channel[3:] == "166"
    ):
        detlist = [
            "001_003_030_00A_" + channel[3:] + "_T",
            "001_003_030_00A_" + channel[3:] + "_B",
        ]
    elif (
        instrument == "MFT"
        and channel[3:] == "119"
        or channel[3:] == "166"
    ):
        detlist = [
            "001_001_026_15A_" + channel[3:] + "_T",
            "001_001_026_15A_" + channel[3:] + "_B",
        ]
    elif (
        instrument == "LFT"
        and channel[3:] == "040"
        or channel[3:] == "060"
        or channel == "L1-078"
    ):
        detlist = [
            "000_000_008_UA_" + channel[3:] + "_T",
            "000_000_008_UA_" + channel[3:] + "_B",
        ]
    elif (
        instrument == "LFT"
        and channel[3:] == "050"
        or channel == "L2-068"
        or channel == "L2-089"
    ):
        detlist = [
            "000_000_002_UA_" + channel[3:] + "_T",
            "000_000_002_UA_" + channel[3:] + "_B",
        ]
    elif (
        instrument == "LFT"
        and channel == "L3-068"
        or channel == "L3-089"
        or channel[3:] == "119"
    ):
        detlist = [
            "000_001_035_UB_" + channel[3:] + "_T",
            "000_001_035_UB_" + channel[3:] + "_B",
        ]
    elif (
        instrument == "LFT"
        and channel == "L4-078"
        or channel[3:] == "100"
        or channel[3:] == "140"
    ):
        detlist = [
            "000_001_017_QB_" + channel[3:] + "_T",
            "000_001_017_QB_" + channel[3:] + "_B",
        ]
    elif (
        instrument == "HFT"
        and channel[3:] == "235"
        or channel[3:] == "337"
    ):
        detlist = [
            "002_001_069_Q_" + channel[3:] + "_T",
            "002_001_069_Q_" + channel[3:] + "_B",
        ]
    elif (
        instrument == "HFT"
        and channel[3:] == "195"
        or channel[3:] == "280"
    ):
        detlist = [
            "002_000_120_Q_" + channel[3:] + "_T",
            "002_000_120_Q_" + channel[3:] + "_B",
        ]
    elif instrument == "HFT" and channel[3:] == "402":
        detlist = [
            "002_002_000_Q_" + channel[3:] + "_T",
            "002_002_000_Q_" + channel[3:] + "_B",
        ]

    else:
        raise RuntimeError("No list of detector provided!")
    return detlist

In [3]:
syst = sbm.Systematics()
freq_maps = []
freq_maps_input = []
freq_maps_res = []
# fg_models = ["pysm_dust_1", "pysm_synch_1", "pysm_freefree_1", "pysm_ame_1", "pysm_co_1"]
fg_models = ["pysm_dust_1", "pysm_synch_1", "pysm_co_1"]

# adding gaussian noise on the bandpasses
sigma_band = 0.05

Mbsparams = lbs.MbsParameters(
    make_cmb=False,
    cmb_r=0.0,
    make_fg=True,
    seed_cmb=1234,
    fg_models=fg_models,
    gaussian_smooth=False,
    bandpass_int=False,
    nside=128,
    units="uK_CMB",
    maps_in_ecliptic=False,
)

np.random.seed(0)
for channel in sbm.channel_list:
    config = sbm.Configlation(imo, channel)
    config.mdim = 2  # 2
    config.parallel = True
    config.nside = 128
    config.xlink_threshold = 0.7  # avoid mapmake in singular pixels
    config.use_hwp = False

    # Define the telescope
    telescope = channel[0] + "FT"

    # Load the channel info
    ch_info = lbs.FreqChannelInfo.from_imo(
        url=f"/releases/{imo_version}/satellite/{telescope}/{channel}/channel_info",
        imo=imo,
    )

    # detectors = ch_info.detector_names[:6]
    detectors = detector_list(telescope, channel)

    dets = []
    for n_det in detectors:
        det = lbs.DetectorInfo.from_imo(
            url=f"/releases/{imo_version}/satellite/{telescope}/{channel}/{n_det}/detector_info",
            imo=imo,
        )
        dets.append(det)

    bandcenter_ghz = dets[0].bandcenter_ghz
    bandwidth_ghz = dets[0].bandwidth_ghz

    band_low_edge = bandcenter_ghz - bandwidth_ghz / 2.0
    band_high_edge = bandcenter_ghz + bandwidth_ghz / 2.0

    # generating different bandpasses for each detector
    for di, det in enumerate(detectors):
        Band = lbs.BandPassInfo(
            bandcenter_ghz=bandcenter_ghz,
            bandwidth_ghz=0.8
            * bandwidth_ghz,  # to have the wings cross the band extremes at ~0.5
            bandlow_ghz=max(0, band_low_edge - 50),
            bandhigh_ghz=band_high_edge + 50,
            bandtype="cheby",
            normalize=False,
            nsamples_inband=len(
                np.arange(max(0, band_low_edge - 50), band_high_edge + 50, 1)
            ),  # bandpass sampled from center +- 2width,
            # to have a 0.5 GHz resol. we should ask from 4*width points
            cheby_ripple_dB=0.2,
            cheby_poly_order=3,
        )

        err_inband = sigma_band * np.random.normal(0.0, 1.0, Band.weights.size)
        Band.weights += err_inband

        Band.freqs_ghz = Band.freqs_ghz[Band.weights**2 >= 0.01]
        Band.weights = (
            (Band.weights[Band.weights**2 >= 0.01]) ** 2
            / Band.freqs_ghz
            / Band.freqs_ghz
            / 1e18
        )

        dets[di].band_freqs_ghz = Band.freqs_ghz
        dets[di].band_weights = Band.weights

    syst.set_bandpass_mismatch(detectors)

    output, input_map = sbm.sim_bandpass_mismatch(
        config,
        syst,
        Mbsparams,
        dets,
    )
    freq_maps.append(output)
    freq_maps_input.append(input_map)
    freq_maps_res.append(output - input_map)

freq_maps_m2 = np.array(freq_maps)
freq_maps_input = np.array(freq_maps_input)
freq_maps_res = np.array(freq_maps_res)

Processing L3-068: 100%|██████████| 1/1 [00:01<00:00,  1.27s/it]


CO line 115.271 in the band
CO line 115.271 in the band


Processing L4-078: 100%|██████████| 1/1 [00:01<00:00,  1.26s/it]


CO line 115.271 in the band


Processing L1-078: 100%|██████████| 1/1 [00:01<00:00,  1.28s/it]


...

Processing H1-280: 100%|██████████| 1/1 [00:01<00:00,  1.25s/it]


CO line 345.796 in the band
CO line 345.796 in the band


Processing H2-337: 100%|██████████| 1/1 [00:01<00:00,  1.19s/it]


CO line 345.796 in the band
CO line 345.796 in the band


Processing H3-402: 100%|██████████| 1/1 [00:01<00:00,  1.24s/it]
